<a href = "https://colab.research.google.com/github/flyingzaptop/pyton-colab/blob/main/Main/reges% target =" _ pary "> <mmg src =" https://colab.research.google.com/assets/colab-badge.svg "alt =" otwórz się w kolab "/https://colab.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/House_Rent_Dataset.csv", index_col="Unnamed: 0")

[Informacje o danych] (https://www.kaggle.com/datasets/iamsouravbanerjee/house-rent-prediction-dataset)

In [3]:
df.head()

,Posted On,BHK,Rent,Size,Floor,Area Type,Area Locality,City,Furnishing Status,Tenant Preferred,Bathroom,Point of Contact
0,2022-05-18,2.0,10000.0,1100.0,Ground out of 2,NaN,NaN,Kolkata,Unfurnished,Bachelors/Family,2.0,Contact Owner
1,2022-05-13,2.0,20000.0,800.0,1 out of 3,Super Area,"Phool Bagan, Kankurgachi",Kolkata,Semi-Furnished,Bachelors/Family,1.0,NaN
2,2022-05-16,2.0,17000.0,1000.0,1 out of 3,Super Area,Salt Lake City Sector 2,Kolkata,Semi-Furnished,Bachelors/Family,1.0,Contact Owner
3,2022-07-04,2.0,10000.0,800.0,1 out of 2,Super Area,Dumdum Park,Kolkata,Unfurnished,Bachelors/Family,1.0,Contact Owner
4,NaN,2.0,7500.0,850.0,1 out of 2,Carpet Area,NaN,Kolkata,Unfurnished,Bachelors,1.0,Contact Owner


In [4]:
df = df[['Size', 'Furnishing Status', 'City', 'Rent']]
df.head()

,Size,Furnishing Status,City,Rent
0,1100.0,Unfurnished,Kolkata,10000.0
1,800.0,Semi-Furnished,Kolkata,20000.0
2,1000.0,Semi-Furnished,Kolkata,17000.0
3,800.0,Unfurnished,Kolkata,10000.0
4,850.0,Unfurnished,Kolkata,7500.0


# Zadanie 1
Podaj główne cechy danych liczbowych, podziel na szkolenia i czaty testowe. Ukierunkowany znak `Rent '

In [10]:
df.isna().sum()

,0
Size,344
Furnishing Status,351
City,358
Rent,377


In [11]:
df.dropna(inplace=True)

In [12]:
from sklearn.model_selection import train_test_split
# Основні числові характеристики даних
print(df.describe())
X = df.drop('Rent', axis=1)
y = df['Rent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

              Size          Rent
count  3477.000000  3.477000e+03
mean    970.483175  3.453861e+04
std     627.121627  8.146501e+04
min      20.000000  1.200000e+03
25%     565.000000  1.000000e+04
50%     850.000000  1.600000e+04
75%    1200.000000  3.300000e+04
max    7000.000000  3.500000e+06
X_train shape: (2781, 3)
X_test shape: (696, 3)
y_train shape: (2781,)
y_test shape: (696,)


# Zadanie 2
Utwórz rurociąg do przetwarzania kolumn numerycznych (wypełnienie luk)

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
numerical_features = ['Size']
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean'))
])


# Zadanie 3
Utwórz rurociąg, aby przetwarzać kategoryczne kolumny (wypełnianie luk i kodowanie)

In [14]:
from sklearn.preprocessing import OneHotEncoder
categorical_features = ['Furnishing Status', 'City']
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Zadanie 4
Połącz poprzedni rurociąg z jednym kolumnowym transformerem

In [15]:
from sklearn.compose import ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Zadanie 5
Utwórz ostateczny model i przeszkolić go

In [16]:
from sklearn.linear_model import LinearRegression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Size']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Furnishing Status',
                                                   'City'])])),
                ('regressor', LinearRegression())])

# Zadanie 6
Wydaj podstawowe wskaźniki modelu

In [17]:
from sklearn.metrics import mean_squared_error, r2_score
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

Mean Squared Error: 1275992658.902451
R-squared: 0.5396771985793151


# Zadanie 7
Zapisz model, pobierz go i poprzedzaj cenę samochodu

In [19]:
data = pd.DataFrame({'Size': [900],
                     'Furnishing Status': ['Semi-Furnished'],
                     'City': ['Mumbai']})

data

,Size,Furnishing Status,City
0,900,Semi-Furnished,Mumbai


In [20]:
import pickle
filename = 'rent_prediction_model.pkl'
pickle.dump(model, open(filename, 'wb'))
loaded_model = pickle.load(open(filename, 'rb'))
rent_prediction = loaded_model.predict(data)

print(f"Прогнозована орендна плата: {rent_prediction[0]}")


Прогнозована орендна плата: 81874.18305288901
